In [103]:
from elasticsearch import Elasticsearch
es = Elasticsearch('http://localhost:9200')
import pandas as pd
from tqdm import tqdm


! netstat -a -n | grep tcp | grep 9200 

tcp6       0      0  ::1.9200               ::1.65006              ESTABLISHED
tcp6       0      0  ::1.65006              ::1.9200               ESTABLISHED
tcp4       0      0  127.0.0.1.9200         *.*                    LISTEN     
tcp6       0      0  ::1.9200               *.*                    LISTEN     


# Opening File

In [3]:
df = pd.read_csv("movies_metadata.csv").dropna()
df.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0
68,False,"{'id': 43563, 'name': 'Friday Collection', 'po...",3500000,"[{'id': 35, 'name': 'Comedy'}]",http://www.newline.com/properties/friday.html,10634,tt0113118,en,Friday,Craig and Smokey are two guys in Los Angeles h...,...,1995-04-26,28215918.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A lot can go down between thursday and saturda...,Friday,False,7.0,513.0
69,False,"{'id': 10924, 'name': 'From Dusk Till Dawn Col...",19000000,"[{'id': 27, 'name': 'Horror'}, {'id': 28, 'nam...",http://www.miramax.com/movie/from-dusk-till-dawn/,755,tt0116367,en,From Dusk Till Dawn,Seth Gecko and his younger brother Richard are...,...,1996-01-19,25836616.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,One night is all that stands between them and ...,From Dusk Till Dawn,False,6.9,1644.0
153,False,"{'id': 439053, 'name': 'Brooklyn Cigar Store C...",2000000,"[{'id': 35, 'name': 'Comedy'}]",http://miramax.com/movie/blue-in-the-face/,5894,tt0112541,en,Blue in the Face,"Auggie runs a small tobacco shop in Brooklyn, ...",...,1995-09-15,1275000.0,83.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Welcome to the planet Brooklyn.,Blue in the Face,False,6.8,28.0
178,False,"{'id': 286162, 'name': 'Power Rangers Collecti...",15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.powerrangers.com/,9070,tt0113820,en,Mighty Morphin Power Rangers: The Movie,Power up with six incredible teens who out-man...,...,1995-06-30,66000000.0,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Power Is On!,Mighty Morphin Power Rangers: The Movie,False,5.2,153.0


In [6]:
df = pd.read_csv("movies_metadata.csv").dropna()
df[['title','overview','release_date','vote_average']]

,title,overview,release_date,vote_average
9,GoldenEye,James Bond must unmask the mysterious head of ...,1995-11-16,6.6
68,Friday,Craig and Smokey are two guys in Los Angeles h...,1995-04-26,7.0
69,From Dusk Till Dawn,Seth Gecko and his younger brother Richard are...,1996-01-19,6.9
153,Blue in the Face,"Auggie runs a small tobacco shop in Brooklyn, ...",1995-09-15,6.8
178,Mighty Morphin Power Rangers: The Movie,Power up with six incredible teens who out-man...,1995-06-30,5.2
...,...,...,...,...
44274,War for the Planet of the Apes,Caesar and his apes are forced into a deadly c...,2017-07-11,6.7
44674,Goon: Last of the Enforcers,"During a pro lockout, Doug ""The Thug"" Glatt is...",2017-03-17,6.0
44821,Pokémon: Spell of the Unknown,When Molly Hale's sadness of her father's disa...,2000-07-08,6.0
44842,Transformers: The Last Knight,"Autobots and Decepticons are at war, with huma...",2017-06-21,6.2


# Mapping Source

In [101]:
source = {
  "settings": {
    "number_of_shards": 2,
    "number_of_replicas": 1,
    "refresh_interval": "1m"},
  "properties": {
        "title": {
        "type": "text",                
          },      
        "overview": {
        "type": "text",                
          },
        "date": {
        "type": "release_date",                
        "format": "yyyy-MM-dd"},
        "vote_average": {
        "type": "integer",                
          }          
        }
      }

# Create ElasticSearch Index

In [102]:
index_name = 'test-index_movie'
resp = es.indices.create(index=index_name,body=source)

# Adding to Index

In [105]:
title = list(df['title'])
overview = list(df['overview'])
release_date = list(df['release_date'])
vote_average = list(df['vote_average'])

for i in tqdm(range(len(df))):
    es.index(index='test-index_movie', id=i, body={
                                               'title':title[i],
                                               'overview':overview[i],
                                               'release_date': release_date[i],
                                               'vote_average':vote_average[i]})
    
    
    

100%|██████████| 693/693 [00:03<00:00, 208.21it/s]


# Adding a decay rate for older Movies

In [99]:
res = es.search(index='test-index_movie',size=5,body={
            "query": {
                "function_score": { "query": {
                    "match": { 'overview': 'war'}
                },
                    "gauss": {
                        "release_date": {
                              "origin": 'now', 
                              "scale": "60d",
                              "offset": "3d", 
                              "decay" : '0.90'
                        }
                    }
                }
            }
    }
)

for hit in res['hits']['hits']:
    print('Title:',hit['_source']['title'])
    print('Date:',hit['_source']['release_date'])
    print('Score:',hit['_score'])
   

Title: Transformers: The Last Knight
Date: 2017-06-21
Score: 3.085726e-16
Title: Underworld: Blood Wars
Date: 2016-11-28
Score: 1.1334413e-22
Title: Batman v Superman: Dawn of Justice
Date: 2016-03-23
Score: 5.524216e-32
Title: The Hunger Games: Mockingjay - Part 2
Date: 2015-11-18
Score: 2.6088054e-37
Title: Terminator Genisys
Date: 2015-06-23
Score: 6.4e-44


In [65]:
result = es.search(index='test-index_movie',size=10,
                   body={"query": { "match_phrase_prefix" : {"overview" : {"query":"war"}}},
                         "sort":[{"_score": { "order": "desc" }},{"release_date" : {"order" : "desc"}}]}
                   )
for hit in res['hits']['hits']:
    print('Title:',hit['_source']['title'])
    print('Date:',hit['_source']['release_date'])
    print('Score:',hit['_score'])

Title: Transformers: The Last Knight
Date: 2017-06-21
Score: 3.3065226
Title: Underworld: Blood Wars
Date: 2016-11-28
Score: 2.9025762
Title: Justice League: Throne of Atlantis
Date: 2015-01-27
Score: 2.6151757
Title: Batman v Superman: Dawn of Justice
Date: 2016-03-23
Score: 2.3509247
Title: X-Men: Days of Future Past
Date: 2014-05-15
Score: 2.3507874


In [48]:
field = 'overview'
phrase = 'war'
source = {'query':{'match':{field:{"query": phrase}}}}


res = es.search(index='test-index_movie', body = source,size=5)
    
for hit in res['hits']['hits']:
    print('Title:',hit['_source']['title'])
    print('Date:',hit['_source']['release_date'])
    print('Score:',hit['_score'])

Title: Underworld
Date: 2003-09-19
Score: 3.8601506
Title: Rambo
Date: 2008-01-24
Score: 3.8585582
Title: Why We Fight: Prelude to War
Date: 1942-05-27
Score: 3.7823527
Title: Captain America: The First Avenger
Date: 2011-07-22
Score: 3.707629
Title: Cats & Dogs 2 : The Revenge of Kitty Galore
Date: 2010-07-30
Score: 3.670098


In [9]:
source = {"query": {
            "more_like_this" : {
            "fields" :['overview'],
            "like" : [{"_id" : 10}],
            "min_term_freq" : 1,
            "min_doc_freq":1,
            "max_query_terms" : 15,
            "minimum_should_match":'30%'}}}

res = es.search(index='test-index_movie', body = source,size=5)
    
for hit in res['hits']['hits']:
    print('Date:',hit['_source']['release_date'])
    print('Score:',hit['_score'])
    print('Title:',hit['_source']['title'])
    

Date: 2005-02-08
Score: 10.97977
Title: Constantine
Date: 2010-10-04
Score: 10.317138
Title: 30 Days of Night: Dark Days
Date: 2016-10-20
Score: 10.175927
Title: Ouija: Origin of Evil
Date: 2008-07-01
Score: 9.891312
Title: The Mummy: Tomb of the Dragon Emperor
Date: 2015-06-23
Score: 8.765725
Title: Terminator Genisys


In [87]:
index_name = 'test-index_movie'
start_date = '2016-01-31'
end_date = '2016-12-31' 
size = 5
source = {"query": {"bool": {"must": 
                    [{"range": {"release_date": {
                    "gte": start_date, 
                    "lte": end_date,}}}]}}}

res = es.search(index = index_name,body = source, size = size)

for hit in res['hits']['hits']:
    print('Date:',hit['_source']['release_date'])
    print('Title:',hit['_source']['title'])


Date: 2016-02-09
Title: Deadpool
Date: 2016-04-27
Title: Captain America: Civil War
Date: 2016-03-09
Title: Allegiant
Date: 2016-06-22
Title: Independence Day: Resurgence
Date: 2016-03-23
Title: Batman v Superman: Dawn of Justice


# Using phrases, finding matching phrases

In [109]:
source = {"query": {
            "more_like_this" : {
            "fields" :['overview'],
            "like" : ["war"],
            "min_term_freq" : 1,
            "min_doc_freq":1,
            "max_query_terms" : 12,
            "minimum_should_match":'20%'}}}

res = es.search(index='test-index_movie', body = source,size=5)
    
for hit in res['hits']['hits']:
    print('Title:',hit['_source']['title'])
    print('Date:',hit['_source']['release_date'])
    print('Score:',hit['_score'])

Title: Underworld
Date: 2003-09-19
Score: 3.8601506
Title: Rambo
Date: 2008-01-24
Score: 3.8585582
Title: Why We Fight: Prelude to War
Date: 1942-05-27
Score: 3.7823527
Title: Captain America: The First Avenger
Date: 2011-07-22
Score: 3.707629
Title: Cats & Dogs 2 : The Revenge of Kitty Galore
Date: 2010-07-30
Score: 3.670098


In [124]:
source = {'query':
          {'match':
           {'title':
            {"query": 'warz',"fuzziness":1}}},
          'highlight': {'fields': {'title':{}}}}


res = es.search(index='test-index_movie', body = source,size=5)
    
for i in range(len(res['hits']['hits'])):
    print(res['hits']['hits'][i]['highlight']['title'][0])
    print(res['hits']['hits'][i]['_source']['release_date'])

Library <em>Wars</em>
2013-04-26
Star <em>Wars</em>
1977-05-25
Underworld: Blood <em>Wars</em>
2016-11-28
World <em>War</em> Z
2013-06-20
Justice League: <em>War</em>
2014-02-02


Library <em>Wars</em>

2013-04-26

Star <em>Wars</em>

1977-05-25

Underworld: Blood <em>Wars</em>

2016-11-28

World <em>War</em> Z

2013-06-20

Justice League: <em>War</em>

2014-02-02

# Reference:

[Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset)

[Elastic Search Python](https://elasticsearch-py.readthedocs.io/en/master/)

[Function Score](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-function-score-query.html)

[More Like This](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-mlt-query.html)

[Date Range](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-daterange-aggregation.html)